# Cassandra Study 4-Data modeling 2

## Query-Driven Modeling Practice

In [37]:
import cassandra
import prettytable as pt

In [3]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)    

In [4]:
try:
    query = "select * from hotels_by_poi"
    session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="No keyspace has been specified. USE a keyspace, or explicitly specify keyspace.tablename"


In [5]:
try:
    session.execute("""
    create keyspace if not exists miketest
    with replication=
    {'class':'SimpleStrategy','replication_factor':1
    }""")
except Exception as e:
    print(e)    

In [6]:
try:
    session.set_keyspace('miketest')
except Exception as e:
    print(e)        

### Defining application queries

Q1. Find hotels near a given point of interest.

Q2. Find information about a given hotel, such as its name and location.

Q3. Find points of interest near a given hotel.

Q4. Find an available room in a given date range.

Q5. Find the rate and amenities for a room.

Q6. Lookup a reservation by confirmation number.

Q7. Lookup all reservations by guest name.

Q8. Lookup a reservation by hotel, date, and guest name.

Q9. View guest details.

Reference
https://cassandra.apache.org/doc/latest/cassandra/data_modeling/data_modeling_queries.html

### Create Logical data model tables

In [7]:
query = "CREATE TABLE IF NOT EXISTS hotels_by_poi"
query = query + "(poi_name text, hotel_id int, name text, phone int, address text, PRIMARY KEY (poi_name, hotel_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
query = "CREATE TABLE IF NOT EXISTS hotels"
query = query + "(hotel_id int, name text, phone text, address text, PRIMARY KEY (hotel_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
query = "CREATE TABLE IF NOT EXISTS pois_by_hotel"
query = query + "(hotel_id int, poi_name text, description text, PRIMARY KEY (hotel_id, poi_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)  
query = "CREATE TABLE IF NOT EXISTS available_rooms_by_hotel_date"
query = query + "(hotel_id text, date date, room_number int, is_abailable boolean, PRIMARY KEY (hotel_id, date, room_number))"
try:
    session.execute(query)
except Exception as e:
    print(e)
query = "CREATE TABLE IF NOT EXISTS amenities_by_room"
query = query + "(hotel_id text, room_id int, amenity_name text, description text, PRIMARY KEY ((hotel_id, room_id), amenity_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)
query = "CREATE TABLE IF NOT EXISTS reservations_by_confirmation"
query = query + "(confirm_number int, hotel_id int, room_id int, start_date date, end_date date, guest_id int, PRIMARY KEY (confirm_number, hotel_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
query = "CREATE TABLE IF NOT EXISTS reservations_by_guest"
query = query + "(guest_last_name text, hotel_id int, guest_id int, room_id int, start_date date, end_date date, confirm_number int, PRIMARY KEY (guest_last_name, hotel_id, guest_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
query = "CREATE TABLE IF NOT EXISTS reservations_by_hotel_date"
query = query + "(hotel_id int, start_date date, room_id int, end_date date, confirm_number int, gyest_id int, PRIMARY KEY ((hotel_id, start_date), room_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
query = "CREATE TABLE IF NOT EXISTS guests"
query = query + "(guest_id int, first_name text, last_name text, title text, email text, phone_numbers int, address text, PRIMARY KEY (guest_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)    

### The above create table CQL is just for thinking, not the actual tables

In [8]:
query = "drop table hotels_by_poi"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table hotels"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table pois_by_hotel"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table available_rooms_by_hotel_date"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table amenities_by_room"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table reservations_by_confirmation"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table reservations_by_guest"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table reservations_by_hotel_date"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table guests"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)    

### Transform into Physical Data Modeling

In [9]:
try:
    session.execute("""
    create keyspace if not exists hotel
    with replication=
    {'class':'SimpleStrategy','replication_factor':1
    }""")
except Exception as e:
    print(e)    

In [10]:
try:
    session.set_keyspace('hotel')
except Exception as e:
    print(e)  

#### 1. create new column type about address data

In [11]:
query = "CREATE TYPE IF NOT EXISTS address "
query = query + "(street text, city text, state_or_province text, postal_code text, country text)"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### 2. Create hotels table include address type column.

In [12]:
query = "CREATE TABLE IF NOT EXISTS hotels "
query = query + "(id text, name text, phone text, address frozen<address>, PRIMARY KEY (id)) "
query = query + "WITH comment ='Q2. Find information about a hotel'"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### 3. Insert a test row to check if the create table CQL currect and how it display data.

In [21]:
query = "INSERT INTO hotels (id, name, phone, address)"
query = query + " VALUES (%s, %s, %s, {street:%s, city:%s, state_or_province:%s, postal_code:%s, country:%s})"

try:
    session.execute(query, ("A1111", "Crowne Plaza Tainan", "063911899", "289 Zhouping Road, Anping District,","Tainan","Taiwan","70841","Taiwan"))
except Exception as e:
    print(e)

try:
    session.execute(query, ("A1112", "Silks Place Tainan", "062136290", "No. 1, Heyi Rd, West Central District,","Tainan","Taiwan","700","Taiwan"))
except Exception as e:
    print(e)    

In [45]:
query = 'SELECT * FROM hotels'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

t= pt.PrettyTable(['id', 'name', 'phone', 'address'])    
for row in rows:
    t.add_row([row.id, row.name, row.phone, row.address])
print (t)

+-------+---------------------+-----------+------------------------------------------------------------------------------------------------------------------------------------------+
|   id  |         name        |   phone   |                                                                 address                                                                  |
+-------+---------------------+-----------+------------------------------------------------------------------------------------------------------------------------------------------+
| A1112 |  Silks Place Tainan | 062136290 | address(street='No. 1, Heyi Rd, West Central District,', city='Tainan', state_or_province='Taiwan', postal_code='700', country='Taiwan') |
| A1111 | Crowne Plaza Tainan | 063911899 | address(street='289 Zhouping Road, Anping District,', city='Tainan', state_or_province='Taiwan', postal_code='70841', country='Taiwan')  |
+-------+---------------------+-----------+------------------------------------------

#### 4. delete test row if you don't want to remain the row

In [20]:
query = "delete from hotels where id = 'A1112'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e) 

#### 5. continue create other physical table.

In [41]:
query = "CREATE TABLE IF NOT EXISTS hotels_by_poi "
query = query + "(poi_name text, hotel_id text, name text, phone text, address frozen<address>, PRIMARY KEY (poi_name, hotel_id)) "
query = query + "WITH comment ='Q1. Find hotels near given poi' and CLUSTERING ORDER BY (hotel_id ASC)"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS pois_by_hotel"
query = query + "(hotel_id text, poi_name text, description text, PRIMARY KEY (hotel_id, poi_name))"
query = query + "WITH comment ='Q3. Find pois near a hotel'"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS available_rooms_by_hotel_date "
query = query + "(hotel_id text, date date, room_number smallint, is_abailable boolean, PRIMARY KEY (hotel_id, date, room_number))"
query = query + "WITH comment ='Q4. Find available rooms by hotel date'"
try:
    session.execute(query)
except Exception as e:
    print(e)    
    
query = "CREATE TABLE IF NOT EXISTS amenities_by_room "
query = query + "(hotel_id text, room_id smallint, amenity_name text, description text, PRIMARY KEY ((hotel_id, room_id), amenity_name)) "
query = query + "WITH comment ='Q5. Find amenities for a room'"
try:
    session.execute(query)
except Exception as e:
    print(e)    

#### 6. Insert other rows in other tables.

In [23]:
query = "INSERT INTO hotels_by_poi (poi_name, hotel_id, name, phone, address)"
query = query + " VALUES (%s, %s, %s, %s, {street:%s, city:%s, state_or_province:%s, postal_code:%s, country:%s})"

try:
    session.execute(query, ("Anping Old Fort", "A1111", "Crowne Plaza Tainan", "063911899", "289 Zhouping Road, Anping District,","Tainan","Taiwan","70841","Taiwan"))
except Exception as e:
    print(e)

try:
    session.execute(query, ("Former Tait & Co. Merchant House", "A1111", "Crowne Plaza Tainan", "063911899", "289 Zhouping Road, Anping District,","Tainan","Taiwan","70841","Taiwan"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, ("Tainan Confucius Temple", "A1112", "Silks Place Tainan", "062136290", "No. 1, Heyi Rd, West Central District,","Tainan","Taiwan","700","Taiwan"))
except Exception as e:
    print(e)    
    
try:
    session.execute(query, ("Tainan Art Museum Building 2", "A1112", "Silks Place Tainan", "062136290", "No. 1, Heyi Rd, West Central District,","Tainan","Taiwan","700","Taiwan"))
except Exception as e:
    print(e)        

In [44]:
query = "SELECT * FROM hotels_by_poi WHERE poi_name = 'Anping Old Fort'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

t= pt.PrettyTable(['poi_name', 'hotel_id', 'name', 'phone', 'address'])    
for row in rows:
    t.add_row([row.poi_name, row.hotel_id, row.name, row.phone, row.address])
print (t)

+-----------------+----------+---------------------+-----------+-----------------------------------------------------------------------------------------------------------------------------------------+
|     poi_name    | hotel_id |         name        |   phone   |                                                                 address                                                                 |
+-----------------+----------+---------------------+-----------+-----------------------------------------------------------------------------------------------------------------------------------------+
| Anping Old Fort |  A1111   | Crowne Plaza Tainan | 063911899 | address(street='289 Zhouping Road, Anping District,', city='Tainan', state_or_province='Taiwan', postal_code='70841', country='Taiwan') |
+-----------------+----------+---------------------+-----------+----------------------------------------------------------------------------------------------------------------------------

In [24]:
query = "INSERT INTO pois_by_hotel (hotel_id, poi_name, description)"
query = query + " VALUES (%s, %s, %s)"

try:
    session.execute(query, ("A1111", "Anping Old Fort",  "This 17th-century fortress built on a peninsula by the Dutch East India Company is now a museum."))
except Exception as e:
    print(e)

try:
    session.execute(query, ("A1111", "Former Tait & Co. Merchant House", "Popular site featuring Taiwan history exhibits in a former warehouse overgrown with banyan trees."))
except Exception as e:
    print(e)
    
try:
    session.execute(query, ("A1112", "Tainan Confucius Temple", "Small, 17th-century Confucian temple featuring traditional architecture & a tranquil courtyard."))
except Exception as e:
    print(e)    
    
try:
    session.execute(query, ("A1112", "Tainan Art Museum Building 2", "Striking space for contemporary Taiwanese art & sculpture exhibitions, with a restaurant & cafe."))
except Exception as e:
    print(e)        

In [43]:
query = "SELECT * FROM pois_by_hotel WHERE hotel_id = 'A1112'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

t = pt.PrettyTable(['hotel_id', 'poi_name', 'description'])    
for row in rows:
    t.add_row([row.hotel_id, row.poi_name, row.description])
print (t)

+----------+------------------------------+--------------------------------------------------------------------------------------------------+
| hotel_id |           poi_name           |                                           description                                            |
+----------+------------------------------+--------------------------------------------------------------------------------------------------+
|  A1112   | Tainan Art Museum Building 2 | Striking space for contemporary Taiwanese art & sculpture exhibitions, with a restaurant & cafe. |
|  A1112   |   Tainan Confucius Temple    | Small, 17th-century Confucian temple featuring traditional architecture & a tranquil courtyard.  |
+----------+------------------------------+--------------------------------------------------------------------------------------------------+


In [31]:
query = "INSERT INTO available_rooms_by_hotel_date (hotel_id, date, room_number, is_abailable)"
query = query + " VALUES (%s, %s, %s, %s)"

try:
    session.execute(query, ("A1111", "2022-10-01", 1, True))
except Exception as e:
    print(e)
try:
    session.execute(query, ("A1111", "2022-10-02", 1, True))
except Exception as e:
    print(e)
try:
    session.execute(query, ("A1111", "2022-10-03", 1, False))
except Exception as e:
    print(e)
try:
    session.execute(query, ("A1111", "2022-10-01", 2, False))
except Exception as e:
    print(e)
try:
    session.execute(query, ("A1111", "2022-10-02", 2, True))
except Exception as e:
    print(e)
try:
    session.execute(query, ("A1111", "2022-10-03", 2, True))
except Exception as e:
    print(e)    

In [40]:
query = "SELECT * FROM available_rooms_by_hotel_date WHERE hotel_id = 'A1111' AND date ='2022-10-03'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

t = pt.PrettyTable(['hotel_id', 'date', 'room_number', 'is_abailable'])
for row in rows:
    t.add_row([row.hotel_id, row.date, row.room_number, row.is_abailable]) 
print(t)     

+----------+------------+-------------+--------------+
| hotel_id |    date    | room_number | is_abailable |
+----------+------------+-------------+--------------+
|  A1111   | 2022-10-03 |      1      |    False     |
|  A1111   | 2022-10-03 |      2      |     True     |
+----------+------------+-------------+--------------+


In [46]:
query = "INSERT INTO amenities_by_room (hotel_id, room_id, amenity_name, description)"
query = query + " VALUES (%s, %s, %s, %s)"

try:
    session.execute(query, ("A1111", 1, "TV", "with MOD can select channel you want to watch."))
except Exception as e:
    print(e)

try:
    session.execute(query, ("A1111", 1,  "telephone", "The price must be charged when you call the outside line."))
except Exception as e:
    print(e)
    
try:
    session.execute(query, ("A1111", 2, "TV", "with MOD can select channel you want to watch."))
except Exception as e:
    print(e)

try:
    session.execute(query, ("A1111", 2,  "telephone", "The price must be charged when you call the outside line."))
except Exception as e:
    print(e)

In [47]:
query = "SELECT * FROM amenities_by_room WHERE hotel_id = 'A1111' AND room_id =1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

t = pt.PrettyTable(['hotel_id', 'room_id', 'amenity_name', 'description'])
for row in rows:
    t.add_row([row.hotel_id, row.room_id, row.amenity_name, row.description]) 
print(t)     

+----------+---------+--------------+-----------------------------------------------------------+
| hotel_id | room_id | amenity_name |                        description                        |
+----------+---------+--------------+-----------------------------------------------------------+
|  A1111   |    1    |      TV      |       with MOD can select channel you want to watch.      |
|  A1111   |    1    |  telephone   | The price must be charged when you call the outside line. |
+----------+---------+--------------+-----------------------------------------------------------+


#### If you need to drop table when you test, then below CQL can run

In [ ]:
query = "drop table hotels_by_poi"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table hotels"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table pois_by_hotel"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table available_rooms_by_hotel_date"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table amenities_by_room"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### 7.Create reservation keyspace then Create other tables in it

In [49]:
try:
    session.execute("""
    create keyspace if not exists reservation
    with replication=
    {'class':'SimpleStrategy','replication_factor':1
    }""")
except Exception as e:
    print(e)    

In [50]:
try:
    session.set_keyspace('reservation')
except Exception as e:
    print(e)  

#### 8. Like hotel keyspace to create type and tables about reservations.

In [51]:
query = "CREATE TYPE IF NOT EXISTS address "
query = query + "(street text, city text, state_or_province text, postal_code text, country text)"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [65]:
query = "CREATE TABLE IF NOT EXISTS reservations_by_confirmation "
query = query + "(confirm_number text, hotel_id text, start_date date, end_date date, room_number smallint, guest_id uuid, PRIMARY KEY (confirm_number, hotel_id, start_date, room_number)) "
query = query + "WITH comment ='Q6. Find reservations by confirmation number'"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS reservations_by_hotel_date"
query = query + "(hotel_id text, start_date date, end_date date, room_number smallint, confirm_number text, guest_id uuid, PRIMARY KEY ((hotel_id, start_date), room_number)) "
query = query + "WITH comment ='Q7. Find reservations by hotel and date'"
try:
    session.execute(query)
except Exception as e:
    print(e)    
    
query = "CREATE TABLE IF NOT EXISTS reservations_by_guest "
query = query + "(guest_last_name text, hotel_id text, room_number smallint, start_date date, end_date date, confirm_number text, guest_id uuid, PRIMARY KEY (guest_last_name, hotel_id)) "
query = query + "WITH comment ='Q8. Find reservations by guest name'"
try:
    session.execute(query)
except Exception as e:
    print(e)    
    

    
query = "CREATE TABLE IF NOT EXISTS guests "
query = query + "(guest_id uuid, first_name text, last_name text, title text, emails set<text>, phone_numbers list<text>, address map<text, frozen<address>>, confirm_number text, PRIMARY KEY (guest_id))"
query = query + "WITH comment ='Q9. Find guest by ID'"
try:
    session.execute(query)
except Exception as e:
    print(e)    

#### 9. like test rows insert into tables in hotel keyspace, we just insert test rows in guests table for special column formats, here.

In [66]:
query = "INSERT INTO guests (guest_id, first_name, last_name, title, emails, phone_numbers, address, confirm_number)"
query = query + " VALUES (now(),%s, %s, %s, {%s}, [%s], {%s:{street:%s, city:%s, state_or_province:%s, postal_code:%s, country:%s}}, %s)"

try:
    session.execute(query, ("Joe", "Chen", "Mr.", "test1@gamil.com","022222222", "home", "1 test Road, test District,","Taipei","Taiwan","111","Taiwan","1234"))
except Exception as e:
    print(e)


In [67]:
query = "SELECT * FROM guests"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#for row in rows:
    #print(row.guest_id, row.first_name, row.last_name, row.title, row.emails, row.phone_numbers, row.address, row.confirm_number) 
t = pt.PrettyTable(['guest_id', 'first_name', 'last_name', 'title', 'emails', 'phone_numbers', 'address', 'confirm_number'])
for row in rows:
    t.add_row([row.guest_id, row.first_name, row.last_name, row.title, row.emails, row.phone_numbers, row.address, row.confirm_number]) 
print(t)

+--------------------------------------+------------+-----------+-------+--------------------------------+---------------+-----------------------------------------------------------------------------------------------------------------------------------------+----------------+
|               guest_id               | first_name | last_name | title |             emails             | phone_numbers |                                                                 address                                                                 | confirm_number |
+--------------------------------------+------------+-----------+-------+--------------------------------+---------------+-----------------------------------------------------------------------------------------------------------------------------------------+----------------+
| 78cb6150-4ec2-11ed-b332-7fa5bd83bdad |    Joe     |    Chen   |  Mr.  | SortedSet(['test1@gamil.com']) | ['022222222'] | {'home': address(street='1 test Road, test 

#### If you need to drop table when you test, then below CQL can run

In [64]:
query = "drop table reservations_by_confirmation"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table reservations_by_guest"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table reservations_by_hotel_date"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table guests"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)    

In [50]:
session.shutdown()
cluster.shutdown()